# 토지피복지도 객체 분할

- 이미지 세그멘테이션

In [1]:
!nvidia-smi

Sun Dec 11 07:27:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.11    Driver Version: 525.60.11    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### for Colab

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !unzip /content/drive/MyDrive/Project/kaggle-2nd-mock-competition/landmap/DATA/train.zip -d /content/drive/MyDrive/Project/kaggle-2nd-mock-competition/landmap/DATA

In [4]:
# !unzip /content/drive/MyDrive/Project/kaggle-2nd-mock-competition/landmap/DATA/test.zip -d /content/drive/MyDrive/Project/kaggle-2nd-mock-competition/landmap/DATA

## 필수 라이브러리 설치 및 불러오기

In [5]:
# !pip install segmentation_models_pytorch
# !pip install albumentations
# !pip install opencv-python
# !pip install knockknock
# !pip install ptflops
# !pip install torchsummary

In [6]:
import os
import json
import time
import random
import glob
import gc
from datetime import datetime, timezone, timedelta
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image
from skimage import io

from sklearn.model_selection import train_test_split
# from sklearn.model_selection import StratifiedKFold
import cv2
from tqdm import tqdm
from tqdm import tqdm_notebook as ntqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import Dataset, DataLoader

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_encoder
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.losses import JaccardLoss
import albumentations as A

# from ptflops import get_model_complexity_info
# from IPython.display import SVG
from pprint import pprint
from PIL import Image

from knockknock import discord_sender

In [7]:
print(torch.__version__)

1.13.0


## 경로 & 인자 설정

#### 데이터 경로

In [8]:
# 프로젝트 경로

###
# for colab
###
# PROJECT_DIR = '/content/drive/MyDrive/Project/kaggle-2nd-mock-competition/landmap'

###
# for linux
###
PROJECT_DIR = '/home/ubuntu/ai/landmap/'

In [9]:
os.chdir(PROJECT_DIR)
#데이터 경로
DATA_DIR = os.path.join(PROJECT_DIR, 'DATA') # 모든 데이터가 들어있는 폴더 경로
TRAIN_DIR = os.path.join(DATA_DIR, 'train') # 학습 데이터가 들어있는 폴더 경로
TRAIN_IMG_DIR = os.path.join(TRAIN_DIR, 'images') # 학습 이미지가 들어있는 폴더 경로
TRAIN_MASK_DIR = os.path.join(TRAIN_DIR, 'masks') # 학습 마스크가 들어있는 폴더 경로
TRAIN_CSV_FILE = os.path.join(TRAIN_DIR, 'traindf.csv') # 학습 이미지와 마스크 이름이 들어있는 CSV 경로

### Alert Setting

In [10]:
with open( PROJECT_DIR + 'secrets.json') as f:

    json_data = json.load(f)

webhook_url = json_data["webhook"]

In [11]:
with open( PROJECT_DIR + 'secrets.json') as f:

    json_data = json.load(f)

webhook_url_finish = json_data["webhook_finish"]

#### 데이터 수량 확인:
- n_train = 3930
- n_test = 3930

In [12]:
len(os.listdir(TRAIN_IMG_DIR)) #3930

3930

In [13]:
len(os.listdir(TRAIN_MASK_DIR)) #3930

3930

#### 시드 설정

In [14]:
RANDOM_SEED = 42 #랜덤 시드

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

#### 디바이스 설정

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
print(device)

cuda


## Dataset 정의

##### 데이터 불러오기 및 분할

In [17]:
# 학습 이미지, 마스크 이름 들어있는 CSV 불러와 데이터 프레임으로 저장
entiredf = pd.read_csv(TRAIN_CSV_FILE)

# Train과 Validation 데이터셋으로 나누기
traindf, validdf = train_test_split(entiredf, test_size=0.2, random_state= random.seed(RANDOM_SEED))
traindf = traindf.reset_index(drop=True)
validdf = validdf.reset_index(drop=True)

##### Dataset 클래스 정의

In [18]:
class Datasets(Dataset):
    def __init__(self, df, augmentations, img_dir, mask_dir):
        self.df = df # 이미지와 마스크 이름이 저장된 데이터프레임 
        self.augmentations = augmentations # 학습 전 적용할 augmentation
        self.img_dir = img_dir # 이미지 폴더 경로
        self.mask_dir = mask_dir # 마스크 폴더 경로
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # 데이터 프레임 불러와서 이미지와 마스크 경로 설정
        row = self.df.iloc[idx] # 데이터프레임 행 불러오기
        image_path = os.path.join(self.img_dir,row['img'])
        mask_path = os.path.join(self.mask_dir, row['mask'])
        
        # 이미지와 마스크 불러오기
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = np.expand_dims(mask, axis=-1)
        
        # Augmentation 적용하기
        if self.augmentations:
            data = self.augmentations(image=image, mask=mask)
            image = data['image']
            mask = data['mask']
        
        # PyTorch 인풋 모양에 맞게 이미지와 마스크 모양 변경
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        mask = np.transpose(mask, (2,0,1)).astype(np.float32)
        
        # 이미지 Normalization 0~255 픽셀값 --> 0~1 픽셀값
        image = torch.Tensor(image) / 255.0
        mask = torch.round(torch.Tensor(mask)/255.0)
        
        return image, mask

### Augmentation 함수 정의

In [19]:
def get_train_augs():
    return A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),                           # 이미지 크기 변환 3 * 512 *512
        A.RandomCrop(IMG_CROP_SIZE, IMG_CROP_SIZE),             # Random crop
        # A.RandomRotate90(p=0.5),
        A.HorizontalFlip(p=0.5),                                # 이미지 좌우반전
        A.VerticalFlip(p=0.5),                                   # 이미지 상하반전
    ])

def get_valid_augs():
    return A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE)
    ])

## Model 정의

In [20]:
F = nn.functional

class SegModel(nn.Module):
    def __init__(self):
        super(SegModel, self).__init__()
        
        self.backbone = smp.UnetPlusPlus(
            encoder_name = ENCODER,
            encoder_weights = WEIGHTS,
            in_channels = 3,
            classes = 1,
            activation = None,
            decoder_use_batchnorm = True
        )
    
    def forward(self, images):
        logits = self.backbone(images)
        
        return logits

## Settings

##### 하이퍼파라미터 설정

In [21]:
epochs = 68
BATCH_SIZE = 4
LEARNING_RATE = 1e-8 # lr => 0.0000001
early_stopping = 20
IMG_SIZE = 512
IMG_CROP_SIZE = 416 #208 #416
# WEIGHT_DECAY = 1e-3

##### Model Setting

In [22]:
# Model 설정
ENCODER = 'timm-efficientnet-b5'    #'resnet50' #'timm-efficientnet-b5' # 인코더 모델
WEIGHTS = 'imagenet' # Pre-train에 활용된 데이터셋
# ACTIVATION = None #'sigmoid'
model = SegModel().to(device)

In [23]:
model

SegModel(
  (backbone): UnetPlusPlus(
    (encoder): EfficientNetEncoder(
      (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): Swish()
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): Swish()
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): Swish()
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
              (gate): Sigmoid()
            )
            (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn2): BatchNorm2d(2

In [24]:
from torchsummary import summary as summary_
summary_(model, (3, IMG_SIZE, IMG_SIZE), batch_size= BATCH_SIZE)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [4, 48, 256, 256]           1,296
       BatchNorm2d-2          [4, 48, 256, 256]              96
             Swish-3          [4, 48, 256, 256]               0
            Conv2d-4          [4, 48, 256, 256]             432
       BatchNorm2d-5          [4, 48, 256, 256]              96
             Swish-6          [4, 48, 256, 256]               0
            Conv2d-7              [4, 12, 1, 1]             588
             Swish-8              [4, 12, 1, 1]               0
            Conv2d-9              [4, 48, 1, 1]             624
          Sigmoid-10              [4, 48, 1, 1]               0
    SqueezeExcite-11          [4, 48, 256, 256]               0
           Conv2d-12          [4, 24, 256, 256]           1,152
      BatchNorm2d-13          [4, 24, 256, 256]              48
         Identity-14          [4, 24, 2

In [25]:
# freeze

# for param in model.parameters():
#     param.requires_grad = False
    
    
# model.aux_classifier = None # default값

In [26]:
# modelclassifier = torchvision.models.segmentation.deeplabv3plus.DeepLabV3PlusHead(512, 1)

##### Optimizer &  Loss 설정

In [27]:
# # ssl 인증 오류 시
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [28]:
# optimizer 설정
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# 학습 loss funciton 설정
loss_fn = DiceLoss(mode = 'binary')

In [29]:
# Scheduler
from torch.optim import lr_scheduler

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max = 200,eta_min = 1e-6)

## Dataset & Dataloader 생성

In [30]:
# Dataset 및 Dataloader 설정

train_dataset = Datasets(traindf, get_train_augs(), TRAIN_IMG_DIR, TRAIN_MASK_DIR)
valid_dataset = Datasets(validdf, get_valid_augs(), TRAIN_IMG_DIR, TRAIN_MASK_DIR)

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers = 2)
valid_loader = DataLoader(valid_dataset, batch_size = BATCH_SIZE, num_workers = 2)

## Train & Vaild 정의

### Train

In [31]:
# @discord_sender(webhook_url=webhook_url)
def train_fn(model, 
             dataloader, 
             loss_fn, 
             optimizer,
             ):
    model.train()
    
    train_total_loss = 0.0
    tp_t, fp_t, fn_t, tn_t = [], [], [], []
    dataset_size = 0
    
    status_bar = tqdm(dataloader, leave=True,)
    
    for images,masks in status_bar:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        logits = model(images)
        loss = loss_fn(logits, masks)
        
        # tp, fp, fn, tn = smp.metrics.get_stats(output, target, mode='binary')
        pred_mask = (logits > 0.5).float()
        tp, fp, fn, tn = smp.metrics.get_stats(pred_mask.long(), masks.long(), mode='binary')
        
        
        loss.backward()
        optimizer.step()
        
        # if scheduler is not None:
        #     scheduler.step()
        
        # For Checking
        
        ds = images.shape[0]
        dataset_size += ds
        # train_total_loss += loss.item()
        # train_epoch_loss = train_total_loss / len(dataloader)
        train_total_loss += (loss.item() *  ds)
        train_epoch_loss = train_total_loss / dataset_size
        
        
        tp_t.append(tp)
        fp_t.append(fp)
        fn_t.append(fn)
        tn_t.append(tn)
        
        tp = torch.cat(tp_t)
        fp = torch.cat(fp_t)
        fn = torch.cat(fn_t)
        tn = torch.cat(tn_t)
        
        # accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
        # precision = smp.metrics.precision(tp, fp, fn, tn, reduction="micro")
        # f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
        # recall = smp.metrics.recall(tp, fp, fn, tn, reduction="micro")
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro-imagewise")
        dataset_iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        
        # status_bar.set_postfix(
        status_bar.set_description(
            f"T_Epoch_Loss:{train_epoch_loss:.5f} || iou_score:{iou_score:.5f} || DataSet_iou_score:{dataset_iou_score:.5f}")
        
        # for epoch in tqdm(dataloader, desc="info", leave=True):
        #     print(f"||T_Epoch_Loss:{train_epoch_loss} || iou_score:{iou_score} || DataSet_iou_score:{dataset_iou_score}")            
    
    ##########################################################################################
    
    metrics = dict()
    metrics['loss'] = train_epoch_loss
    # metrics['accuracy'] = accuracy.detach().cpu().item()
    # metrics['precision'] = precision.detach().cpu().item()
    # metrics['f1_score'] = f1_score.detach().cpu().item()
    # metrics['recall'] = recall.detach().cpu().item()
    metrics['per_iou_score'] = iou_score.detach().cpu().item()
    metrics['dataset_iou_score'] = dataset_iou_score.detach().cpu().item()
    
    
    return metrics

### Validation 함수

In [32]:
# @discord_sender(webhook_url=webhook_url)
@torch.no_grad()
def valid_fn(model, 
             dataloader, 
             loss_fn
             ):
    
    model.eval()
    
    valid_total_loss = 0.0
    dataset_size = 0
    tp_v, fp_v, fn_v, tn_v = [], [], [], []
    
    status_bar = tqdm(dataloader, leave=True,)
    
    with torch.no_grad():
        for images,masks in status_bar:
            images = images.to(device)
            masks = masks.to(device)
            
            logits = model(images)
            loss = loss_fn(logits, masks)
            
            # y_pred = logits / y_true = masks
            # tp, fp, fn, tn = smp.metrics.get_stats(output, target, mode='binary')
            pred_mask = (logits > 0.5).float()
            tp, fp, fn, tn = smp.metrics.get_stats(pred_mask.long(), masks.long(), mode='binary')
            
            
            
            # For Checking
            
            ds = images.shape[0]
            dataset_size += ds
            # valid_total_loss += loss.item()
            # valid_epoch_loss = valid_total_loss / len(dataloader)
            valid_total_loss += (loss.item() *  ds)
            valid_epoch_loss = valid_total_loss / dataset_size
            
            # torch.cat(tensors, dim=0, *, out=None) → Tensor
            tp_v.append(tp)
            fp_v.append(fp)
            fn_v.append(fn)
            tn_v.append(tn)
            
            tp = torch.cat(tp_v)
            fp = torch.cat(fp_v)
            fn = torch.cat(fn_v)
            tn = torch.cat(tn_v)
            
            # accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
            # precision = smp.metrics.precision(tp, fp, fn, tn, reduction="micro")
            # f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
            # recall = smp.metrics.recall(tp, fp, fn, tn, reduction="micro")
            iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro-imagewise")
            dataset_iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
            
            # tqdm(dataloader).set_description(
            #     f'|| V_Epoch_Loss:{valid_epoch_loss} || ACC:{accuracy} || f1_score:{f1_score} || iou_score:{iou_score} || DataSet_iou_score:{dataset_iou_score}'
            # )
            
            # status_bar.set_postfix(
            status_bar.set_description(
            f"T_Epoch_Loss:{valid_epoch_loss:.5f} || iou_score:{iou_score:.5f} || DataSet_iou_score:{dataset_iou_score:.5f}")
            
            # for epoch in tqdm(dataloader, desc="info", leave=True):
            #     print(f"||V_Epoch_Loss:{valid_epoch_loss} || iou_score:{iou_score} || DataSet_iou_score:{dataset_iou_score}")  
    
    ##########################################################################################
    
    metrics = dict()
    metrics['loss'] = valid_epoch_loss
    # metrics['accuracy'] = accuracy.detach().cpu().item()
    # metrics['precision'] = precision.detach().cpu().item()
    # metrics['f1_score'] = f1_score.detach().cpu().item()
    # metrics['recall'] = recall.detach().cpu().item()
    metrics['per_iou_score'] = iou_score.detach().cpu().item()
    metrics['dataset_iou_score'] = dataset_iou_score.detach().cpu().item()
    
    
    return metrics

## 결과 저장 경로 설정

In [33]:
# 파일명에 현재 모델명 및 파라메터명 추가
# file_name = inspect.getfile(inspect.currentframe())
file_name = f'Encoder={ENCODER}_epoch={str(epochs)}_bach={str(BATCH_SIZE)}_lr={str(LEARNING_RATE)}_imagesize={str(IMG_SIZE)}'


# 시간 고유값 
kst = timezone(timedelta(hours=9))        
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S_") + file_name


# 기록 경로
RECORDER_DIR = os.path.join(PROJECT_DIR, 'results', 'train', train_serial)
# 현재 시간 기준 폴더 생성
os.makedirs(RECORDER_DIR, exist_ok=True)    

In [34]:
print(RECORDER_DIR)

/home/ubuntu/ai/landmap/results/train/20221211_162728_Encoder=timm-efficientnet-b5_epoch=68_bach=4_lr=1e-08_imagesize=512


## Trainer

In [35]:
def run_trainer(model, 
                 loss_fn, 
                 train_loader,
                 valid_loader,
                 optimizer,
                 epochs, 
                 early_stopping,
                 ):
    
    print(f"epoch: {epochs} || early_stopping: {early_stopping}")
    
    if torch.cuda.is_available():
        print("GPU_INFO : {}\n".format(torch.cuda.get_device_name()))
        
    start = time.time()
    
    best_epoch = np.inf
    best_loss = np.inf
    best_score = 0
    
    train_logs, valid_logs = [],  []
    train_recall, valid_recall = [],  []
    train_precision_score, valid_precision_score = [],  []
    train_accuracy_score, valid_accuracy_score = [],  []
    train_f1_score, valid_f1_score = [],  []
    train_per_iou_score, valid_per_iou_score = [], []
    train_dataset_iou_score, valid_dataset_iou_score = [], []
    
    print_iter = 1
    
    for epoch in range(0,epochs):
        
        gc.collect()
        
        print(f"epoch : {epoch} / {epochs}")
        train_metrics = train_fn(model,
                                 train_loader,
                                 loss_fn,
                                 optimizer,
                                )
                                       
        gc.collect() # Garbage Collection
        
        valid_metrics = valid_fn(model,
                                 valid_loader,
                                 loss_fn,
                                )
        
##############################################################################################
        
        train_logs += [train_metrics['loss']]
        valid_logs += [valid_metrics['loss']]
        # train_recall += [train_metrics['recall']]
        # valid_recall += [valid_metrics['recall']]
        # train_precision_score += [train_metrics['precision']]
        # valid_precision_score += [valid_metrics['precision']]
        # train_accuracy_score += [train_metrics['accuracy']]
        # valid_accuracy_score += [valid_metrics['accuracy']]
        # train_f1_score += [train_metrics['f1_score']]
        # valid_f1_score += [valid_metrics['f1_score']]
        train_per_iou_score += [train_metrics['per_iou_score']]
        valid_per_iou_score += [valid_metrics['per_iou_score']]
        # train_dataset_iou_score += [train_metrics['dataset_iou_score']]
        # valid_dataset_iou_score += [valid_metrics['dataset_iou_score']]
        
        print()
        
        if (epoch) % print_iter == 0:
            # print(f"Epoch:{epoch} \n||T_Epoch_Loss:{train_metrics['loss']} | V_Epoch_Loss:{valid_metrics['loss']}\n||ACC:{valid_metrics['accuracy']}\n||f1_score:{valid_metrics['f1_score']}\n||iou_score:{valid_metrics['per_iou_score']}\n||DataSet_iou_score:{valid_metrics['dataset_iou_score']}\n")
            print(f"Epoch:{epoch} || T_Epoch_Loss:{train_metrics['loss']} || V_Epoch_Loss:{valid_metrics['loss']} || iou_score:{valid_metrics['per_iou_score']} || DataSet_iou_score:{valid_metrics['dataset_iou_score']}")
            
            
        # if best_score < valid_metrics['per_iou_score']:
        #     print(f"Validation per_iou_score Highest score achieved \n best_score: {best_score} \n >>> new best_score: {valid_metrics['per_iou_score']:.4f}")
            
        #     best_score = valid_metrics['per_iou_score']
        #     torch.save(model.state_dict(), os.path.join(RECORDER_DIR, "best-iou-score-model.pt"))
        #     print('new best model saved')
        #     print()
            
        if valid_metrics['loss']< best_loss:
            print(f"Validation Loss Highest score achieved \n best_loss: {best_loss} \n >>> new best_loss: {valid_metrics['loss']:.4f}")
            best_loss = valid_metrics['loss']
            best_epoch = epoch
            torch.save(model.state_dict(), os.path.join(RECORDER_DIR, "best-loss-model.pt"))
            print('new best model saved')
            print()
            
            
            print()
        
        else:
            early_stopping += 1
        
        # Early Stopping
        if early_stopping == 10:
            print()
            print("###################### early_stopping reached 10 ######################")
            break
        
        
    
    print()
    print()
    end = time.time()
    total_process_time = end - start
    print(f"best_loss: {best_loss} at {best_epoch}Epoch")
    print(f"Total Process Time:{total_process_time // 3600:.0f}h {(total_process_time % 3600) // 60:.0f}m")
    print('<< Finished Training>>')
    
    result = dict()
    result["Train Loss"] = train_logs
    result["Valid Loss"] = valid_logs
    
    # result["Train Recall"] = train_recall
    # result["Valid Recall"] = valid_recall
    
    # result["Train Precision"] = train_precision_score
    # result["Valid Precision"] = valid_precision_score
    
    # result["Train Accuracy"] = train_accuracy_score
    # result["Valid Accuracy"] = valid_accuracy_score
    
    # result["Train F1 Score"] = train_f1_score
    # result["Valid F1 Score"] = valid_f1_score
    
    result["Train per Image IOU"] = train_per_iou_score
    result["Valid per Image IOU"] = valid_per_iou_score
    
    result["Train Dataset IOU"] = train_dataset_iou_score
    result["Valid Dataset IOU"] = valid_dataset_iou_score
    
    return model, result

## 학습 진행

In [36]:
model, result = run_trainer(model= model,
                            loss_fn= loss_fn,
                            train_loader = train_loader,
                            valid_loader = valid_loader,
                            optimizer= optimizer,
                            epochs= epochs,
                            early_stopping= early_stopping,
                            )

epoch: 68 || early_stopping: 20
GPU_INFO : Tesla T4

epoch : 0 / 68


T_Epoch_Loss:0.8832654962494115 || iou_score:0.03802020847797394 || DataSet_iou_score:0.04964694753289223:  40%|███▉      | 314/786 [03:12<04:45,  1.65it/s]  

In [ ]:
## Train/Valid Loss History
plot_from = 0
plt.figure(figsize=(20, 10))
plt.title("Train/Valid Loss Logs", fontsize = 20)
plt.plot(
    range(0, len(result['Train Loss'][plot_from:])), 
    result['Train Loss'][plot_from:], 
    label = 'Train Loss'
    )

plt.plot(
    range(0, len(result['Valid Loss'][plot_from:])), 
    result['Valid Loss'][plot_from:], 
    label = 'Valid Loss'
    )

plt.legend()
# plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
# ## Train/Valid Accuracy History
# plot_from = 0
# plt.figure(figsize=(20, 10))
# plt.title("Train/Valid Accuracy History", fontsize = 20)
# plt.plot(
#     range(0, len(result['Train Accuracy'][plot_from:])), 
#     result['Train Accuracy'][plot_from:], 
#     label = 'Train Accuracy'
#     )

# plt.plot(
#     range(0, len(result['Valid Accuracy'][plot_from:])), 
#     result['Valid Accuracy'][plot_from:], 
#     label = 'Valid Accuracy'
#     )

# plt.legend()
# # plt.yscale('log')
# plt.grid(True)

In [ ]:
# ## Train/Valid Recall History
# plot_from = 0
# plt.figure(figsize=(20, 10))
# plt.title("Train/Valid Recall History", fontsize = 20)
# plt.plot(
#     range(0, len(result['Train Recall'][plot_from:])), 
#     result['Train Recall'][plot_from:], 
#     label = 'Train Recall'
#     )

# plt.plot(
#     range(0, len(result['Valid Recall'][plot_from:])), 
#     result['Valid Recall'][plot_from:], 
#     label = 'Valid Recall'
#     )

# plt.legend()
# # plt.yscale('log')
# plt.grid(True)

In [ ]:
# ## Train/Valid Precision History
# plot_from = 0
# plt.figure(figsize=(20, 10))
# plt.title("Train/Valid Precision History", fontsize = 20)
# plt.plot(
#     range(0, len(result['Train Precision'][plot_from:])), 
#     result['Train Precision'][plot_from:], 
#     label = 'Train Precision'
#     )

# plt.plot(
#     range(0, len(result['Valid Precision'][plot_from:])), 
#     result['Valid Precision'][plot_from:], 
#     label = 'Valid Precision'
#     )

# plt.legend()
# # plt.yscale('log')
# plt.grid(True)

In [ ]:
# ## Train/Valid F1 History
# plot_from = 0
# plt.figure(figsize=(20, 10))
# plt.title("Train/Valid F1 Score History", fontsize = 20)
# plt.plot(
#     range(0, len(result['Train F1 Score'][plot_from:])), 
#     result['Train F1 Score'][plot_from:], 
#     label = 'Train F1 Score'
#     )

# plt.plot(
#     range(0, len(result['Valid F1 Score'][plot_from:])), 
#     result['Valid F1 Score'][plot_from:], 
#     label = 'Valid F1 Score'
#     )

# plt.legend()
# # plt.yscale('log')
# plt.grid(True)

In [ ]:
## Train/Valid Per Image IOU History
plot_from = 0
plt.figure(figsize=(20, 10))
plt.title("Train/Valid per Image IOU History", fontsize = 20)
plt.plot(
    range(0, len(result['Train per Image IOU'][plot_from:])), 
    result['Train per Image IOU'][plot_from:], 
    label = 'Train per Image IOU'
    )

plt.plot(
    range(0, len(result['Valid per Image IOU'][plot_from:])), 
    result['Valid per Image IOU'][plot_from:], 
    label = 'Valid per Image IOU'
    )

plt.legend()
# plt.yscale('log')
plt.grid(True)

In [ ]:
## Train/Valid Dataset IOU History
plot_from = 0
plt.figure(figsize=(20, 10))
plt.title("Train/Valid Dataset IOU History", fontsize = 20)
plt.plot(
    range(0, len(result['Train Dataset IOU'][plot_from:])), 
    result['Train Dataset IOU'][plot_from:], 
    label = 'Train Dataset IOU'
    )

plt.plot(
    range(0, len(result['Valid Dataset IOU'][plot_from:])), 
    result['Valid Dataset IOU'][plot_from:], 
    label = 'Valid Dataset IOU'
    )

plt.legend()
# plt.yscale('log')
plt.grid(True)

## 추론 - Evaluation

#### 마스크를 RLE 형태로 변환해주는 함수

In [ ]:
def mask_to_rle(mask):
    flatten_mask = mask.flatten()
    if flatten_mask.max() == 0:
        return f'0 {len(flatten_mask)}'
    idx = np.where(flatten_mask!=0)[0]
    steps = idx[1:]-idx[:-1]
    new_coord = []
    step_idx = np.where(np.array(steps)!=1)[0]
    start = np.append(idx[0], idx[step_idx+1])
    end = np.append(idx[step_idx], idx[-1])
    length = end - start + 1
    for i in range(len(start)):
        new_coord.append(start[i])
        new_coord.append(length[i])
    new_coord_str = ' '.join(map(str, new_coord))
    return new_coord_str

#### Test 데이터셋 불러오기

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df, img_dir):
        self.df = df
        self.img_dir = img_dir
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        imname = row['img']
        image_path = os.path.join(self.img_dir,imname)
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        image = torch.Tensor(image) / 255.0
        
        return image,imname

#### 경로 및 기타 인자 설정

In [ ]:
TEST_DIR = os.path.join(DATA_DIR, 'test') # 테스트 데이터가 들어있는 폴더 경로
TEST_IMG_DIR = os.path.join(TEST_DIR, 'images') # 테스트 이미지가 들어있는 폴더 경로
TEST_CSV_FILE = os.path.join(TEST_DIR, 'testdf.csv') # 테스트 이미지 이름이 들어있는 CSV 경로

#### 테스트 Dataset, DataLoader 설정

In [ ]:
testdf = pd.read_csv(TEST_CSV_FILE)
test_dataset = TestDataset(testdf, TEST_IMG_DIR)
test_loader = DataLoader(dataset=test_dataset, batch_size=1,shuffle=False, num_workers=2)

#### 최고 성능 모델 불러오기 (loss)

In [ ]:
model.load_state_dict(torch.load(os.path.join(RECORDER_DIR, 'best-loss-model.pt')))
# model.load_state_dict(torch.load(os.path.join('', 'best-model.pt'))) # 끊긴 경우

#### 추론 진행 (loss)

In [ ]:
file_list = [] # 이미지 이름 저장할 리스트
pred_list = [] # 마스크 저장할 리스트
class_list = [] # 클래스 이름 저장할 리스트 ('building')

model.eval()
with torch.no_grad():
    for batch_index, (image,imname) in tqdm(enumerate(test_loader)):
        image = image.to(DEVICE)
        logit_mask = model(image)
        pred_mask = torch.sigmoid(logit_mask) # logit 값을 probability score로 변경
        pred_mask = (pred_mask > 0.5) * 1.0 # 0.5 이상 확률 가진 픽셀값 1로 변환
        pred_rle = mask_to_rle(pred_mask.detach().cpu().squeeze(0)) # 마스크를 RLE 형태로 변경
        pred_list.append(pred_rle)
        file_list.append(imname[0])
        class_list.append("building")
        

#### 예측 결과 파일 만들기

In [ ]:
# 예측 결과 데이터프레임 만들기
results = pd.DataFrame({'img_id':file_list,'class':class_list,'prediction':pred_list})

# sample_submission.csv와 같은 형태로 변형
sampledf = pd.read_csv(os.path.join(TEST_DIR, 'sample_submission.csv'))
sorter = list(sampledf['img_id'])
results = results.set_index('img_id')
results = results.loc[sorter].reset_index()
                       
# 결과 저장
results.to_csv(os.path.join(RECORDER_DIR, 'loss-prediction.csv'), index=False)

## iou

#### 최고 성능 모델 불러오기 (iou)

In [ ]:
model.load_state_dict(torch.load(os.path.join(RECORDER_DIR, 'best-iou-model.pt')))
# model.load_state_dict(torch.load(os.path.join('', 'best-model.pt'))) # 끊긴 경우

#### 추론 진행 (loss)

In [ ]:
file_list = [] # 이미지 이름 저장할 리스트
pred_list = [] # 마스크 저장할 리스트
class_list = [] # 클래스 이름 저장할 리스트 ('building')

model.eval()
with torch.no_grad():
    for batch_index, (image,imname) in tqdm(enumerate(test_loader)):
        image = image.to(DEVICE)
        logit_mask = model(image)
        pred_mask = torch.sigmoid(logit_mask) # logit 값을 probability score로 변경
        pred_mask = (pred_mask > 0.5) * 1.0 # 0.5 이상 확률 가진 픽셀값 1로 변환
        pred_rle = mask_to_rle(pred_mask.detach().cpu().squeeze(0)) # 마스크를 RLE 형태로 변경
        pred_list.append(pred_rle)
        file_list.append(imname[0])
        class_list.append("building")
        

#### 예측 결과 파일 만들기

In [ ]:
# 예측 결과 데이터프레임 만들기
results = pd.DataFrame({'img_id':file_list,'class':class_list,'prediction':pred_list})

# sample_submission.csv와 같은 형태로 변형
sampledf = pd.read_csv(os.path.join(TEST_DIR, 'sample_submission.csv'))
sorter = list(sampledf['img_id'])
results = results.set_index('img_id')
results = results.loc[sorter].reset_index()
                       
# 결과 저장
results.to_csv(os.path.join(RECORDER_DIR, 'iou-prediction.csv'), index=False)

In [ ]:
#### 최고 성능 모델 불러오기 (loss)
model.load_state_dict(torch.load(os.path.join(RECORDER_DIR, 'best-iou-model.pt')))


In [ ]:
# alert()

## 참고자료 


- https://www.kaggle.com/code/ligtfeather/semantic-segmentation-is-easy-with-pytorch
- https://light-tree.tistory.com/216
- http://www.gisdeveloper.co.kr/?p=8443
-